In [2]:
import sys
sys.path.insert(0,'../models/')
sys.path.insert(0,'../datasets/')
sys.path.insert(0,'..')

import pandas as pd
import numpy as np
import json
from subprocess import Popen, PIPE, STDOUT
import re
from collections import defaultdict

import tensorflow as tf
import tqdm
from models import PropbankEncoder
import config

INPUT_DIR = '../datasets/binaries/'
PROPBANK_GLO50_PATH = '{:}deep_glo50.pickle'.format(INPUT_DIR)


<h1><center>Structured Y Network CWIS SRL (BR)</center></h1>

<center>In this notebook we solve the semantic role labeling task using structured predictions networks.</center>

## 1. Builds a "human friendly" version of the dataset

In [3]:
dfgs = pd.read_csv('../datasets/csvs/gs.csv', index_col=0, sep=',', encoding='utf-8')
column_files = [
    '../datasets/csvs/column_chunks/chunks.csv',
    '../datasets/csvs/column_predmarker/predicate_marker.csv',
    '../datasets/csvs/column_shifts_ctx_p/form.csv',
    '../datasets/csvs/column_shifts_ctx_p/gpos.csv',
    '../datasets/csvs/column_shifts_ctx_p/lemma.csv',
    '../datasets/csvs/column_t/t.csv',
    '../datasets/csvs/column_iob/iob.csv'
]

for col_f in column_files:
    _df = pd.read_csv(col_f, index_col=0, encoding='utf-8')
    dfgs = pd.concat((dfgs, _df), axis=1)

DISPLAY_COLUMNS = ['ID', 'P', 'FORM', 'ARG', 'T', 
                   'CHUNK_ID', 'CHUNK_START', 'CHUNK_FINISH', 'CHUNK_LEN', 'CHUNK_CANDIDATE_ID']            
dfgs[DISPLAY_COLUMNS].head(33)    

,ID,P,FORM,ARG,T,CHUNK_ID,CHUNK_START,CHUNK_FINISH,CHUNK_LEN,CHUNK_CANDIDATE_ID
INDEX,,,,,,,,,,
0,1,1,Brasília,*,*,1,0,1,1,0
1,2,1,Pesquisa_Datafolha,(A0*,A0,2,1,4,3,35
2,3,1,publicada,*,A0,2,1,4,3,35
3,4,1,hoje,*),A0,2,1,4,3,35
4,5,1,revela,(V*),V,3,4,5,1,126
5,6,1,um,(A1*,A1,4,5,32,27,181
6,7,1,dado,*,A1,4,5,32,27,181
7,8,1,supreendente,*,A1,4,5,32,27,181
8,9,1,:,*,A1,4,5,32,27,181


## 2. Gets encodings

Propbank Encoder holds an indexed version of propbank dataset an answers to FOUR different dataformats: 
* CAT: this is the raw categorical data.
* EMB: tokens are embedding using GloVe embeddings.
* HOT: onehot encoding of the words and tokens.
* IDX: dense indexed representations.

In [25]:
# LOAD ENCODER
propbank_encoder = PropbankEncoder.recover(PROPBANK_GLO50_PATH)
db = propbank_encoder.db
lex2idx = propbank_encoder.lex2idx
idx2lex = propbank_encoder.idx2lex

# FOR TEXTUAL DATA ONLY
lex2tok = propbank_encoder.lex2tok
tok2idx = propbank_encoder.tok2idx
embeddings = propbank_encoder.embeddings

n_targets = len(lex2idx['T'])

In [5]:
print('attributes\t',
       len(db),
      '\n',             
      'records\t',
       len(db['ARG'].keys()))

attributes	 44 
 records	 141730


In [6]:
def filter_type(ds_type, db):
    '''Filters only records from train dataset
    '''
    ds_types = ('train', 'test', 'valid')
    if ds_type not in ds_types:
        _msg = 'ds_type must be in {:} got {:}'
        _msg = _msg.format(ds_types, ds_type)
        raise ValueError(_msg)
    elif ds_type in ('train',):
        lb = 0 
        ub = config.DATASET_TRAIN_SIZE
    elif ds_type in ('test',):        
        lb = config.DATASET_TRAIN_SIZE
        ub = lb + config.DATASET_VALID_SIZE         
    elif ds_type in ('valid',):                
        lb = config.DATASET_TRAIN_SIZE + config.DATASET_VALID_SIZE
        ub = lb + config.DATASET_TEST_SIZE         

    sel_keys_ = {key_ for key_, prop_ in db['P'].items() if prop_ > lb and prop_ <= ub}

    return {
                attr_:{ idx_: i_
                        for idx_, i_ in dict_.items() if idx_ in sel_keys_
                      }        
                for attr_, dict_  in db.items()
            }

def make_propositions_dict(db):
    '''Reindex db by propositions creating a nested dict in which the
        outer key is the proposition        
    '''
    
    triple_list = []
    prev_prop = -1
    for idx, prop in db['P'].items():
        if prev_prop != prop:
            if idx > 0:
                ub = idx-1
                triple_list.append((lb, ub, prev_prop))
            lb = idx
        prev_prop = prop
    triple_list.append((lb, ub, prev_prop))
            

        
    prop_set = set(db['P'].values())
    return { prop_:
                    {
                        attr_:{ idx_: dict_[idx_]
                                for idx_ in range(lb_, ub_ + 1, 1)
                          }        
                        for attr_, dict_ in db.items() if attr_ not in ('P',)
                    }
             for lb_, ub_, prop_ in  triple_list
            }, {prop_: ub_ - lb_ + 1 for lb_, ub_, prop_ in  triple_list}   


def numpfy_propositions_dict(prop_dict, proplen_dict):
    '''Converts inner dict examples into numpy arrays
    '''
    prop_dict_ = defaultdict(dict)    
    for prop, columns_dict in prop_dict.items():
        len_ = proplen_dict[prop]
        shape_ = (len_, 1)
        for column, values_dict in columns_dict.items():
            tuple_list = [idx_value 
                          for idx_value in values_dict.items()]
            
            tuple_list = sorted(tuple_list, key=lambda x: x[0])            
            # Converts lexicon (raw/indexed) into token (embedded/indexed)
            if (('FORM' in column) or ('LEMMA' in column)):
                values_list = [tok2idx[lex2tok[idx2lex[column][tuple_[1]]]]                
                                   for tuple_ in tuple_list]
            else:
                values_list = [tuple_[1] for tuple_ in tuple_list]
            
            prop_dict_[prop][column]  = np.array(values_list).reshape(shape_)
    
    return prop_dict_        


In [7]:
traindb  = filter_type('train', db)
print('attributes\t',
       len(traindb),
      '\n',             
      'records\t',
       len(traindb['ARG'].keys()),
       '\n',             
      'vocab\t',
        max([form for _, form in traindb['FORM'].items()]))

attributes	 44 
 records	 123846 
 vocab	 13289


In [8]:
prop_dict, proplen_dict = make_propositions_dict(traindb)
print('attributes\t',
       len(prop_dict[1]) + 1,
      '\n',             
      'records\t',
       sum([len(d['ARG']) for p, d in prop_dict.items()]),
        '\n',             
      'vocab\t',
        max([form for _, prop in prop_dict.items() for _, form in prop['FORM'].items()]))

attributes	 44 
 records	 123837 
 vocab	 13289


In [9]:
prop_dict1 = numpfy_propositions_dict(prop_dict, proplen_dict)
print('attributes\t',
       len(prop_dict1[1]) + 1,
      '\n',             
      'records\t',
       sum([len_ for _, len_ in proplen_dict.items()]),
        '\n',             
      'vocab\t',
        max([max(form) for _, prop in prop_dict1.items() for form in prop['FORM']]))

attributes	 44 
 records	 123837 
 vocab	 12037


In [32]:
def get_inputs(db1, propid):
    '''Generate inputs
    '''
    propdb = db1[propid] # nested dict of columns and idx value
    proplen = len(propdb['ID'])
    if 'CHUNK_SPACE' not in propdb:
        propdb['CHUNK_SPACE'] = generate_chunk_space(proplen)

    word    = propdb['FORM']
    ctx_p_left  = propdb['FORM_CTX_P-1']
    ctx_p0  = propdb['FORM_CTX_P+0']
    ctx_p_right  = propdb['FORM_CTX_P+1']
    
    marker  = propdb['MARKER']
    pos     = propdb['GPOS']
    chunk_type  = propdb['T']
    chunk_start, chunk_finish = propdb['CHUNK_SPACE']
    
    return word, ctx_p_left, ctx_p0, ctx_p_right, marker, pos, chunk_type, chunk_start, chunk_finish
            
def generate_chunk_space(n):
    '''Generates all possible spaces for chunks
    '''
    start_list = []
    end_list = []
    for i in range(n):
        for j in range(i,n,1):
            start_list.append(i)
            end_list.append(j+1)
    shape_ = (len(start_list), 1)
    start_ = np.array(start_list).reshape(shape_)
    finish_ = np.array(end_list).reshape(shape_)
    return start_, finish_
            

# def get_outputs(db1, propid):
#     ''' Generate outputs
#     '''
#     propdb_ = db1[propid] # nested dict of columns and idx value
#     plen_ = len(propdb_['ID'])
#     if 'OUTPUTS' not in propdb_: 
#         propdb_['OUTPUTS'] = propdb_['T'].reshape((plen_,))

#     return propdb_['OUTPUTS']

def get_outputs(db1, propid, n_targets):
    ''' Generate outputs
    '''
    propdb = db1[propid] # nested dict of columns and idx value
    if 'OUTPUTS' not in propdb: 
        # FIX THIS!!!
        chunk_list = zip(propdb['CHUNK_CANDIDATE_ID'].values(), 
                         propdb['T'].values())
        chunk_list = list(set(chunk_list))

        propdb['OUTPUTS'] = [ i * n_targets + j for i, j in chunk_list]

    return propdb['OUTPUTS']

In [33]:
%%timeit
propid = 1120
word, ctx_p_left, ctx_p0, ctx_p_right, marker, pos, chunk_type, chunk_start, chunk_finish = get_inputs(prop_dict1, propid)
y = get_outputs(prop_dict1, propid, n_targets)
# worst proposition 1120 size 92!

AttributeError: 'numpy.ndarray' object has no attribute 'values'

 ## MODEL

In [31]:
propid = 1
word, ctx_p_left, ctx_p0, ctx_p_right, marker, pos, chunk_type, chunk_start, chunk_finish = get_inputs(prop_dict1,  propid)
y = get_outputs(prop_dict1, propid, n_targets)
# proplen = proplen_dict[propid]
# y = y.reshape((proplen,1))
print(y)
_start  = np.repeat(chunk_start, n_targets)
_finish = np.repeat(chunk_finish, n_targets)
print([(_start[y_].flatten(), _finish[y_].flatten()) for y_ in y])
print(list(zip(_start[y].flatten(), _finish[y].flatten())))


[ 0  1  1  1 35  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  0]
[(array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1])), (array([0]), array([1]))]
[(0, 1), (0, 1), (0, 1), (0, 1), (0, 1)

In [16]:
import struct_perc.colored_weighted_interval_scheduling as cwis
import struct_perc.weighted_interval_scheduling as wis
import struct_perc.utils as spu

 ## Tensorflow Graph

In [13]:
# vocab_size = len(lex2idx['FORM']) + 1
# embed_size = 50

# n_pos = len(lex2idx['GPOS'])
# # n_type = len(lex2idx['T'])
# n_classes  = len(lex2idx['T'])

# tf.reset_default_graph()

# # word index and gpos 
# tf_words = tf.placeholder(tf.int32, shape=(None,1))
# tf_pos = tf.placeholder(tf.int32, shape=(None,1))
# # t_x_type = tf.placeholder(tf.int32, shape=(None,1))

# # índices de inicio de intervalo
# tf_s = tf.placeholder(tf.int32, shape=(None,1))
# # índices de fim de intervalo
# tf_f = tf.placeholder(tf.int32, shape=(None,1))

# # replicamos os indicies de inicio e fim para cada classe de chunk possivel
# tf_sc = tf.reshape(
#       tf.tile(tf_s,  [1, n_classes]), [-1,1])
# tf_fc = tf.reshape(
#       tf.tile(tf_f,  [1, n_classes]), [-1,1])

# # n_features = (embed_size + n_pos + n_type)
# n_features = (embed_size + n_pos)
# # hidden_features = 300
# W_shape = (n_features, n_classes)
# EMBS = tf.constant(embeddings)
# # tf_token = tf.Variable(initial_value=None, expected_shape=(embed_size,), dtype=tf.float32, trainable=False)

# # geramos os paramteros do modelo
# with tf.variable_scope("model"):
#     W = tf.Variable(
#         tf.random_normal(W_shape, 0, 1/np.sqrt(n_features * n_classes), name='W')
#     )
#     b = tf.Variable(
#         tf.random_normal((n_classes,), 0, 1/np.sqrt(n_classes), name='b')
#     )
    

# # tf_token = tf.nn.embedding_lookup(tf_embeddings, id) 
# # Recuperamos os embeddings de cada palavra
# tf_word_features = tf.gather_nd(EMBS, tf_words)

# tf_pos_flat = tf.reshape(tf_pos, [-1])
# tf_pos_features = tf.one_hot(tf_pos_flat, depth=n_pos)

# # t_x_type_flat = tf.reshape(t_x_type,[-1])
# # t_type_features = tf.one_hot(t_x_type_flat, depth=n_type)

# # X = tf.concat((t_word_features,t_pos_features,t_type_features),axis=1)
# tf_tok_features = tf.concat((tf_word_features,tf_pos_features),axis=1)

# # a partir das features do intervalo computamos o score
# tf_scores = tf.matmul(tf_tok_features, W) + b

# tf_pred = tf.argmax(tf_scores, axis=1)


In [107]:
vocab_size = len(tok2idx)
embed_size = 50
n_pos = len(lex2idx['GPOS'])

n_classes = len(lex2idx['T'])

# índices das palavras
X_words = tf.placeholder(tf.int64, shape=(None,1), name='word')


X_ctx_p_left = tf.placeholder(tf.int64, shape=(None,1), name='ctx_p_left')
X_ctx_p = tf.placeholder(tf.int64, shape=(None,1), name='ctx_p0')
X_ctx_p_right = tf.placeholder(tf.int64, shape=(None,1), name='ctx_p_right')

X_pos = tf.placeholder(tf.int64, shape=(None,1), name='gpos')
X_marker = tf.cast( tf.placeholder(tf.int64, shape=(None,1), name='marker'), tf.float32 )
EMBS = tf.Variable(embeddings, trainable=False)

W_shape = (embed_size * 4 + 1 + n_pos, n_classes)
b_shape = (1,n_classes)
# geramos os paramteros do modelo
with tf.variable_scope("model"):
    W_interval = tf.Variable(tf.zeros(W_shape, dtype=tf.float32), name='W')
    b_interval = tf.Variable(tf.zeros(b_shape, dtype=tf.float32), name='b')


# Recuperamos os embeddings de cada palavra
EMBS_words = tf.gather_nd(EMBS, X_words, name='word_features')
EMBS_ctx_pleft = tf.gather_nd(EMBS, X_ctx_p_left, name='EMBS_ctx_pleft')
EMBS_ctx_p0 = tf.gather_nd(EMBS, X_ctx_p, name='EMBS_ctx_p0')
EMBS_ctx_pright = tf.gather_nd(EMBS, X_ctx_p_right, name='EMBS_ctx_pright')

X_pos_flat = tf.reshape(X_pos, [-1], name='gpos_flat')
X_pos_onehot = tf.one_hot(X_pos_flat, depth=n_pos, name='gpos_onehot')

X = tf.concat((EMBS_words, EMBS_ctx_pleft, EMBS_ctx_p0,
               EMBS_ctx_pright, X_pos_onehot, X_marker),
              axis=1, name='X')

# a partir das features do intervalo computamos o score
score_op = tf.matmul(X, W_interval, name='xW') + b_interval

predict_op = tf.argmax(score_op, axis=1, name='Prediction')

 ## Tensorflow test session

In [103]:
propid  = 1
words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos, chunk_type, chunk_start, chunk_finish = get_inputs(prop_dict1, propid)
y = get_outputs(prop_dict1, propid)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x = sess.run(X, feed_dict={
        X_words:words,
        X_ctx_p_left: ctx_p_left,
        X_ctx_p: ctx_p0,
        X_ctx_p_right: ctx_p_right,        
        X_marker: marker,                
        X_pos:gpos                
    })
print(x.shape)

(33, 226)


In [88]:
def pred(sess, x_words, x_ctx_p_left, x_ctx_p0, x_ctx_p_right, x_marker, x_pos):
    result = sess.run(predict_op,feed_dict={
        X_words: x_words,
        X_ctx_p_left: x_ctx_p_left,
        X_ctx_p: x_ctx_p0,
        X_ctx_p_right: x_ctx_p_right,        
        X_marker: x_marker,                
        X_pos:x_pos
    })
    return result

In [116]:
# índices dos intervalos computados pelo Weighted Interval Scheduling
Y = tf.placeholder(tf.int32, shape=(None,), name='predictions')

# índices dos intervalos corretos
T = tf.placeholder(tf.int32, shape=(None,), name='T')
L = tf.placeholder(tf.int32, shape=(), name='L')
I = tf.to_int32(tf.range(L), name='indices')

# score da estrutura predita
ScoreY = tf.gather_nd(score_op, tf.stack((I, Y), -1), name='predicted_score')
# score da estrutura correta
ScoreT = tf.gather_nd(score_op, tf.stack((I, T), -1), name='target_score')

# função de custo do perceptron estruturado
cost_op = tf.reduce_sum(ScoreY) - tf.reduce_sum(ScoreT)

# gradiente descendente no custo do perceptron estruturado
optimizer = tf.train.GradientDescentOptimizer(0.01)
train_op = optimizer.minimize(cost_op)

In [110]:
predictions = pred(sess, words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos)
print(y.shape)
print(predictions)

(33,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


 ## Can it memorize ?

In [117]:
for i in range(10000):
    
    predictions = pred(sess, words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos)
    
    
    _, cost = sess.run([train_op, cost_op], feed_dict={
        X_words: words,
        X_ctx_p_left: ctx_p_left,
        X_ctx_p: ctx_p0,
        X_ctx_p_right: ctx_p_right,        
        X_marker: marker,        
        X_pos: gpos,
        Y: predictions,
        L: proplen_dict[propid],
        T:y})


    if i % 100 == 0:
        err = np.sum(y.flatten() != predictions.flatten())
        acc = 1 - err/len(predictions.flatten())
        print(acc, ' ', cost)
        if acc == 1:
            break

0.0606060606061   0.0
1.0   0.0


 ## The model reproduces the sentence

In [118]:
p = sess.run(predict_op, 
             feed_dict={
                X_words: words,
                X_ctx_p_left: ctx_p_left,
                X_ctx_p: ctx_p0,
                X_ctx_p_right: ctx_p_right,        
                X_marker: marker,        
                X_pos: gpos
             })
print(p)
print()
print(y)

[ 0  1  1  1 35  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  0]

[ 0  1  1  1 35  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
  2  2  2  2  2  2  2  0]


## Training

In [123]:
sess= tf.Session()
sess.run(tf.global_variables_initializer())

epochs = 30
for i in range(epochs):
    total_err = 0
    total_size = 0
    nprops = len(proplen_dict)
    for propid in tqdm.tqdm(range(1, nprops + 1)):
        words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos, chunk_type, chunk_start, chunk_finish = get_inputs(prop_dict1,  propid)
        y = get_outputs(prop_dict1, propid)

        predictions = pred(sess, words, ctx_p_left, ctx_p0, ctx_p_right, marker, gpos)

        _, cost = sess.run([train_op, cost_op], feed_dict={
            X_words:words,
            X_ctx_p_left: ctx_p_left,
            X_ctx_p: ctx_p0,
            X_ctx_p_right: ctx_p_right,        
            X_marker: marker,
            X_pos: gpos,
            Y:predictions,
            L: proplen_dict[propid],
            T:y})
        total_err += np.sum(predictions.flatten()!=y.flatten())
        total_size += len(predictions.flatten())

    print('epoch ', i, ' acc: ', 1 - total_err/total_size)

  1%|▏         | 73/5099 [00:00<00:06, 723.52it/s]

epoch  0  acc:  0.424138181642


  1%|▏         | 76/5099 [00:00<00:06, 751.57it/s]

epoch  1  acc:  0.447644888038


  1%|          | 62/5099 [00:00<00:08, 613.77it/s]

...

epoch  27  acc:  0.467978067944


  2%|▏         | 77/5099 [00:00<00:06, 763.13it/s]

epoch  28  acc:  0.465838158224


100%|██████████| 5099/5099 [00:07<00:00, 721.11it/s]

epoch  29  acc:  0.469019759846
